In [2]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [38]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Char LSTM

Objectif : 

- Apprendre à implémenter un LSTM en pytorch
- Utiliser LSTM +DataLoader
- Tester la différence entre une validation aléatoire & une validation sur période

### I Data

In [3]:
cat =  open('input.txt', 'r')
data = cat.read()
data = list(map(ord, data))

In [4]:
ord_encoder = OrdinalEncoder()
data = ord_encoder.fit_transform(np.array(data).reshape(-1, 1))

In [5]:
encoder= OneHotEncoder()
encoded_data = encoder.fit_transform(data).todense()
encoded_data.shape

/home/remy/.virtualenvs/pytorch_env/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(1115394, 65)

In [179]:
n_train = 10000
n_valid = 10000
n_test = encoded_data.shape[0]-n_train-n_valid

n_param = encoded_data.shape[1]

### II Modèle 

In [192]:
#paramètres

##LSTM
input_size = n_param
output_size = n_param
hidden_size = 512
num_layers = 3
dropout = 0.2
n_histo = 1
temperature= 2.0


##Learning
learning_rate =  0.0001
batch_size = 400
epochs = 10

In [193]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layer_size = hidden_size
        self.n_histo = n_histo
        self.temperature = temperature
        self.lstm = nn.LSTM(input_size,hidden_size = hidden_size, num_layers = num_layers, dropout=dropout)

        self.hidden_cell = (torch.zeros(num_layers, 1,self.hidden_layer_size).to(device),
                            torch.zeros(num_layers, 1 ,self.hidden_layer_size).to(device))

        self.linear = nn.Linear(self.hidden_layer_size, output_size)
        
    def forward(self, input_seq):
        """
        m = input_seq.shape[0]
        l = []
        for i in range(0, m-n_histo):
            l.append(input_seq[i:i+n_histo,:])
        dat = torch.stack(l,1)
         """
        
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(-1,1, n_param), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions /temperature

In [195]:
model = LSTM().cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [196]:
training_set=  encoded_data[:n_train,:]
training_ds = TensorDataset(torch.tensor(training_set[:-1,:], dtype = torch.float32),torch.tensor(data[1:n_train],dtype = torch.int64))
training_dl = DataLoader(training_ds,batch_size=batch_size , shuffle=False)

In [197]:
valid_set=  encoded_data[n_train:n_train+n_valid,:]
X_valid = torch.tensor(valid_set[:-1,:], dtype = torch.float32)
y_valid = torch.tensor(data[n_train+n_histo:n_train+n_valid],dtype = torch.int64)[:,0]
X_train =  torch.tensor(training_set[:-1,:], dtype = torch.float32)
y_train =  torch.tensor(data[n_histo:n_train],dtype = torch.int64)[:,0]

In [198]:
for i in range(epochs):
    
    y_pred_train = model(X_train.to(device))
    single_loss = loss_function(y_pred_train, y_train.to(device))
    y_pred_valid = model(X_valid.to(device))
    valid_loss = loss_function(y_pred_valid, y_valid.to(device))
    
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f} Valid loss: {valid_loss.item():10.8f}')
    
    for seq, labels in training_dl:



        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(num_layers,1,model.hidden_layer_size).to(device), torch.zeros(num_layers,1,model.hidden_layer_size).to(device))
        y_pred = model(seq.to(device))
        single_loss = loss_function(y_pred, labels[:,0].to(device))
        single_loss.backward()
        optimizer.step()
        
    

        


epoch:   0 loss: 4.17813921 Valid loss: 4.17753363
epoch:   1 loss: 3.72119665 Valid loss: 3.75880098
epoch:   2 loss: 3.26991439 Valid loss: 3.39874411
epoch:   3 loss: 3.24033117 Valid loss: 3.37985468
epoch:   4 loss: 3.23191714 Valid loss: 3.37617826
epoch:   5 loss: 3.22818565 Valid loss: 3.37519884
epoch:   6 loss: 3.22606707 Valid loss: 3.37514424
epoch:   7 loss: 3.22461200 Valid loss: 3.37577391
epoch:   8 loss: 3.22362781 Valid loss: 3.37634087
epoch:   9 loss: 3.22295642 Valid loss: 3.37691116


In [199]:
def decode_txt(seq):
    cate = ord_encoder.inverse_transform(encoder.inverse_transform(np.array(seq)))
    txt = ''.join(list(map(chr,cate)))
    return txt

In [187]:
a= decode_txt(seq)
print(a)

e heart of generosity,
And make bold power look pale--they threw their caps
As they would hang them on the horns o' the moon,
Shouting their emulation.

MENENIUS:
What is granted them?

MARCIUS:
Five tribunes to defend their vulgar wisdoms,
Of their own choice: one's Junius Brutus,
Sicinius Velutus, and I know not--'Sdeath!
The rabble should have first unroof'd the city,
Ere so prevail'd with me:


In [188]:
for i in range(30):
    next_chr =  model(seq.to(device))
    arr = F.softmax(next_chr[-1,:]).detach().cpu().numpy()
    i =  np.argmax(arr)
    a = torch.zeros(1,65)
    a[0,i] = 1
    seq = torch.cat([seq, a])

/home/remy/.virtualenvs/pytorch_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


In [190]:
a= decode_txt(seq)
print(a)

e heart of generosity,
And make bold power look pale--they threw their caps
As they would hang them on the horns o' the moon,
Shouting their emulation.

MENENIUS:
What is granted them?

MARCIUS:
Five tribunes to defend their vulgar wisdoms,
Of their own choice: one's Junius Brutus,
Sicinius Velutus, and I know not--'Sdeath!
The rabble should have first unroof'd the city,
Ere so prevail'd with me:                              


In [191]:
a

"e heart of generosity,\nAnd make bold power look pale--they threw their caps\nAs they would hang them on the horns o' the moon,\nShouting their emulation.\n\nMENENIUS:\nWhat is granted them?\n\nMARCIUS:\nFive tribunes to defend their vulgar wisdoms,\nOf their own choice: one's Junius Brutus,\nSicinius Velutus, and I know not--'Sdeath!\nThe rabble should have first unroof'd the city,\nEre so prevail'd with me:                              "

## Quelques tests avec les tenseurs

In [46]:
tt = torch.tensor([[4,1,0],[4,2,0],[3,3,0],[2,4,0],[4,5,0],[4,6,0],[3,7,0],[2,8,0]])
tt

tensor([[4, 1, 0],
        [4, 2, 0],
        [3, 3, 0],
        [2, 4, 0],
        [4, 5, 0],
        [4, 6, 0],
        [3, 7, 0],
        [2, 8, 0]])

In [49]:
h=2
m = tt.shape[0]
l = []
for i in range(0, m-h):
    l.append(tt[i:i+h,:])
torch.stack(l,1)


tensor([[[4, 1, 0],
         [4, 2, 0],
         [3, 3, 0],
         [2, 4, 0],
         [4, 5, 0],
         [4, 6, 0]],

        [[4, 2, 0],
         [3, 3, 0],
         [2, 4, 0],
         [4, 5, 0],
         [4, 6, 0],
         [3, 7, 0]]])